## Lane Detection
#### we aim to tackle the problem of lane detection given a set of images from a car dashboard or videos. you can choose any such video or dataset for this purpose.
#### a) Implement an auto-encoder model that takes the image as input and outputs the images with the lanes marked. You may instead use python along with OpenCV to implement this as well. Your final goal is to detect the two adjacent lanes (one on left and one on right) given an image.
#### b) Once the lanes have been detected, use these lanes as guidelines to decide the steering angle (Steering angle will depend on the center of the captured image and the center of the lanes detected).

## Importing Libraries

In [10]:
import numpy as np
import cv2
import time

## working on Gray Scale images, applying Gaussian Blur & Canny Edge Detector

In [11]:
def color_edges(image): 
    # Apply color filtering to filter out white lane lines
    lower_white = np.array([0, 160, 10])
    upper_white = np.array([255, 255, 255])
    mask = cv2.inRange(image, lower_white, upper_white)
    result = cv2.bitwise_and(image, image, mask = mask)
    
    # Convert image to grayscale, apply threshold, blur and using cannyED for extracting edges
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
    blur = cv2.GaussianBlur(thresh,(3, 3), 11)
    canny = cv2.Canny(blur, 40, 60)
    return canny

## HLS color system
#### computer vision algorithms used on color images are straightforward extensions to algorithms designed for grayscale images, for instance k-means or fuzzy clustering of pixel colors, or canny edge detection. At the simplest, each color component is separately passed through the same algorithm. It is important, therefore, that the features of interest can be distinguished in the color dimensions used. Because the R, G, and B components of an object's color in a digital image are all correlated with the amount of light hitting the object, and therefore with each other, image descriptions in terms of those components make object discrimination difficult. Descriptions in terms of hue/lightness/chroma or hue/lightness/saturation are often more relevant

## Sobel
#### Sobel filter, is used in image processing and computer vision.

In [12]:
def sobel_binary(img, sobel_kernel=7, mag_thresh=(3, 255), s_thresh=(170, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    gray = hls[:, :, 1]
    s_channel = hls[:, :, 2]
    # Binary matrixes creation
    sobel_binary = np.zeros(shape=gray.shape, dtype=bool)
    s_binary = sobel_binary
    combined_binary = s_binary.astype(np.float32)
    # Sobel Transform
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = 0 
    sobel_abs = np.abs(sobelx**2 + sobely**2)
    sobel_abs = np.uint8(255 * sobel_abs / np.max(sobel_abs))
    sobel_binary[(sobel_abs > mag_thresh[0]) & (sobel_abs <= mag_thresh[1])] = 1
    # Threshold color channel
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Combine the two binary thresholds
    combined_binary[(s_binary == 1) | (sobel_binary == 1)] = 1
    combined_binary = np.uint8(255 * combined_binary / np.max(combined_binary))
    return combined_binary

## Region of interest
#### we determine a region of interest in the form of isosceles trapezoidal and discard any lines outside of this polygon. We assume that the camera remains in the same place across all these image, and lanes are flat, therefore we can identify the critical region we are interested in.

In [13]:
def region_of_interest(img):
    mask = np.zeros_like(img)
    imshape=img.shape
    vertices = np.array([[(150,imshape[0]),(590, 440), (680, 440), (imshape[1]-20,imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, 255)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

# Finding Lanes And Drawing The Lane Area

## Perspective Transform
#### We now need to define a trapezoidal region in the 2D image that will go through a perspective transform to convert into a bird’s eye view. This transform changes the view from car front view to car top view (bird view). This transform keeps straight lines straight

In [14]:
def warp(img, src, dst):
    
    src = np.float32([src])
    dst = np.float32([dst])
    
    return cv2.warpPerspective(img, cv2.getPerspectiveTransform(src, dst),
                               dsize=img.shape[0:2][::-1], flags=cv2.INTER_LINEAR)


## Histogram & Sliding Windows

In [15]:
def sliding_windown(img_w):

    histogram = np.sum(img_w[int(img_w.shape[0] / 2):, :], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((img_w, img_w, img_w)) * 255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0] / 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(img_w.shape[0] / nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img_w.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img_w.shape[0] - (window + 1) * window_height
        win_y_high = img_w.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 2)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 2)
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (
            nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (
            nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    return left_fit, right_fit

##### performing the above process again on the new warped binary image for finding robust line pixels  

In [16]:
def fit_from_lines(left_fit, right_fit, img_w):
    # Assume you now have a new warped binary image
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = img_w.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0] * (nonzeroy ** 2) + left_fit[1] * nonzeroy + left_fit[2] - margin)) & (
    nonzerox < (left_fit[0] * (nonzeroy ** 2) + left_fit[1] * nonzeroy + left_fit[2] + margin)))
    right_lane_inds = (
    (nonzerox > (right_fit[0] * (nonzeroy ** 2) + right_fit[1] * nonzeroy + right_fit[2] - margin)) & (
    nonzerox < (right_fit[0] * (nonzeroy ** 2) + right_fit[1] * nonzeroy + right_fit[2] + margin)))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    return left_fit, right_fit

## Inverse Perspective Transform
#### Finally, we draw the inside the of the lane in green and unwarp the image, thus moving from bird’s eye view to the original undistorted image.

## Radius of curvature
#### compute the lane curvature by calculating the radius of the smallest circle that could be a tangent to our lane lines. Draw a circle that closely fits nearby points on a local section of a curve and curves have the same tangent and curvature at the point where they meet. This radius changes as we move along the curve and on a straight lane the radius would be quite big. The radius of curvature of the curve at a particular point is defined as the radius of the approximating circle.

## Angle of curvature 
#### The degree of curvature is defined as the central angle to the ends of an arc or chord of agreed length. Various lengths are commonly used in different areas of practice. This angle is also the change in forward direction as that portion of the curve is traveled. Where degree of curvature is based on 100 units of arc length, the conversion between degree of curvature and radius is Dr = 18000/π ≈ 5729.57795, where D is degree and r is radius.


In [17]:
def draw_lines(img, img_w, left_fit, right_fit, perspective):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(img_w).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    #color_warp_center = np.dstack((warp_zero, warp_zero, warp_zero))

    ploty = np.linspace(0, img.shape[0] - 1, img.shape[0])

    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    #cv2.fillPoly(color_warp_center, np.int_([pts]), (0, 255, 0))
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = warp(color_warp, perspective[1], perspective[0])
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.2, 0)

    color_warp_lines = np.dstack((warp_zero, warp_zero, warp_zero))
    cv2.polylines(color_warp_lines, np.int_([pts_right]), isClosed=False, color=(255, 255, 255), thickness=25)
    cv2.polylines(color_warp_lines, np.int_([pts_left]), isClosed=False, color=(0, 255, 255), thickness=25)
    newwarp_lines = warp(color_warp_lines, perspective[1], perspective[0])

    result = cv2.addWeighted(result, 1, newwarp_lines, 1, 0)

    # ----- Radius Calculation ------ #

    img_height = img.shape[0]
    y_eval = img_height

    ym_per_pix = 30 / 720.  # meters per pixel in y dimension
    xm_per_pix = 3.7 / 700  # meters per pixel in x dimension

    ploty = np.linspace(0, img_height - 1, img_height)
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty * ym_per_pix, left_fitx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, right_fitx * xm_per_pix, 2)

    # Calculate the new radii of curvature
    left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1]) ** 2) ** 1.5) / np.absolute(
        2 * left_fit_cr[0])

    right_curverad = (
                         (1 + (2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[1]) ** 2) ** 1.5) / np.absolute(
        2 * right_fit_cr[0])
    radius1 = round((float(left_curverad) + float(right_curverad))/2.,2)
    
    if left_fitx[0] - left_fitx[-1] > 60:
        curve_direction = 'Left Curve'
        radius=-5729.57795/radius1
    elif left_fitx[-1] - left_fitx[0] > 60:
        curve_direction = 'Right Curve'
        radius=5729.57795/radius1
    else:
        curve_direction = 'Straight'
        radius=5729.57795/radius1

    # ----- Off Center Calculation ------ #

    lane_width = (right_fit[2] - left_fit[2]) * xm_per_pix
    center = (right_fit[2] - left_fit[2]) / 2
    off_left = (center - left_fit[2]) * xm_per_pix
    off_right = -(right_fit[2] - center) * xm_per_pix
    off_center = round((center - img.shape[0] / 2.) * xm_per_pix,2)

    # --- Print text on screen ------ #
    #if radius < 5000.0:
    text = "Angle = %s [degrees]\noffcenter = %s [m]" % (str(radius), str(off_center))

    for i, line in enumerate(text.split('\n')):
        i = 550 + 20 * i
        cv2.putText(result, line, (0,i), cv2.FONT_HERSHEY_DUPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
    return result,radius

In [18]:
def check_side(smoothed_angle):
    if smoothed_angle<-4:
        return "left"
    elif smoothed_angle<-2:
        return "slightly left"
    elif -2<smoothed_angle<1:
        return "straight"
    elif 2>smoothed_angle>1:
        return "slightly right"
    elif smoothed_angle>2:
        return "right"

### Now using all the above functions to perform lane detection and finding steering angle 

In [19]:
# loading a video
cap = cv2.VideoCapture('Lane.mp4')

out_examples = 0
MOV_AVG_LENGTH = 5
smoothed_angle = 0

while(True):
    start=time.time()
    ret, frame = cap.read()
    if ret is True: 
        image = cv2.resize(frame,(1280,720),interpolation=cv2.INTER_AREA)
    else:
        print("Getting no frames")
        break
    #-------------------------Color & Gradient Threshold------------------------ 
    edges = color_edges(image)
    edges2=sobel_binary(image)
    
    A= cv2.addWeighted(edges2,0.7,edges,0.3,0)
    BW1=cv2.bitwise_and(A, edges2)
    img_b=region_of_interest(BW1)

    # ---------------------------- Perspective Transform --------------------------

    src = [585, 457], [700, 457], [1110, img_b.shape[0]], [220, img_b.shape[0]]
    #src = [480, 500], [800, 500], [img_b.shape[1]-50, img_b.shape[0]],  [150, img_b.shape[0]]

    line_dst_offset = 200
    #src = [595, 500], [685, 500],[1110,img_b.shape[0]],[220, img_b.shape[0]]
    dst = [src[3][0] + line_dst_offset, 0], \
          [src[2][0] - line_dst_offset, 0], \
          [src[2][0] - line_dst_offset, src[2][1]], \
          [src[3][0] + line_dst_offset, src[3][1]]
    
    img_w = warp(img_b, src, dst)
    try:
        # using repeatation process of sliding window and binary wrapped lines
        left_fit, right_fit = fit_from_lines(left_fit, right_fit, img_w)
        mov_avg_left = np.append(mov_avg_left,np.array([left_fit]), axis=0)
        mov_avg_right = np.append(mov_avg_right,np.array([right_fit]), axis=0)
        
    except Exception:
        # usage of only sliding window
        left_fit, right_fit = sliding_windown(img_w)
        mov_avg_left = np.array([left_fit])
        mov_avg_right = np.array([right_fit])

    left_fit = np.array([np.mean(mov_avg_left[::-1][:,0][0:MOV_AVG_LENGTH]),
                        np.mean(mov_avg_left[::-1][:,1][0:MOV_AVG_LENGTH]),
                        np.mean(mov_avg_left[::-1][:,2][0:MOV_AVG_LENGTH])])
    right_fit = np.array([np.mean(mov_avg_right[::-1][:,0][0:MOV_AVG_LENGTH]),
                         np.mean(mov_avg_right[::-1][:,1][0:MOV_AVG_LENGTH]),
                         np.mean(mov_avg_right[::-1][:,2][0:MOV_AVG_LENGTH])])
    if mov_avg_left.shape[0] > 1000:
        mov_avg_left = mov_avg_left[0:MOV_AVG_LENGTH]
    if mov_avg_right.shape[0] > 1000:
        mov_avg_right = mov_avg_right[0:MOV_AVG_LENGTH]
        
    #-----------------------------------------steering Angle----------------------------------------------

    final,degrees = draw_lines(frame, img_w, left_fit, right_fit, perspective=[src,dst])        
    # using the previous steering angle the present steering angle is being calculated w.r.t center of frame and angle of curvature of lane  
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)

    cv2.putText(final,F"{check_side(-smoothed_angle)}: {round(-smoothed_angle)} Degrees",(int(1280//2.7),70), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,0),2,cv2.LINE_AA)
    #out.write(final)
    cv2.imshow('front_view', frame)
    cv2.imshow('canny', edges)
    cv2.imshow('ROI', img_b)
    cv2.imshow('Sky_view', img_w)
    cv2.imshow('final', final)
    #cv2.imshow("steering wheel", dst)

    if cv2.waitKey(1) & 0xFF == 27 or cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
